In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf ,plot_predict
from statsmodels.tsa.arima.model import ARIMA
from sklearn import linear_model
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
path = "../Data/series (1).csv"
df = pd.read_csv(path)

In [4]:
df.shape

(2686, 3)

In [5]:
df.head(3)

,CATEGORY,DATE,UNITS_SOLD
0,CATEG-1,2021-08-17,59
1,CATEG-1,2020-09-19,40
2,CATEG-2,2021-10-26,23


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2686 entries, 0 to 2685
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CATEGORY    2686 non-null   object
 1   DATE        2686 non-null   object
 2   UNITS_SOLD  2686 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 63.1+ KB


In [7]:
df.describe()

,UNITS_SOLD
count,2686.000000
mean,531.340283
std,840.944458
min,0.000000
25%,25.000000
50%,62.000000
75%,695.000000
max,7551.000000


In [8]:
df["CATEGORY"].value_counts()

CATEGORY
CATEG-3    920
CATEG-1    887
CATEG-2    879
Name: count, dtype: int64

In [15]:
df["DATE"] = pd.to_datetime(df["DATE"])
df["DATE"]

0      2021-08-17
1      2020-09-19
2      2021-10-26
3      2021-04-27
4      2021-05-05
          ...    
2681   2021-02-18
2682   2020-12-05
2683   2020-02-05
2684   2020-06-05
2685   2021-09-03
Name: DATE, Length: 2686, dtype: datetime64[ns]

In [14]:
df["UNITS_SOLD"]

0        59
1        40
2        23
3        60
4        61
       ... 
2681     78
2682      8
2683      3
2684    106
2685     38
Name: UNITS_SOLD, Length: 2686, dtype: int64